In [ ]:
# imports
import numpy as np
from obspy import UTCDateTime
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.interpolate import interp1d
from scipy.fftpack import next_fast_len
from utils.model_tools_kurama import logheal_llc
from scipy.special import erf, erfc

### Model for earthquake healing
accelerated implementation using low-level callback by Kurama Okubo 

Okubo, K., Delbridge, B. G., & Denolle, M. A. (2024). Monitoring velocity change over 20 years at Parkfield. Journal of Geophysical Research: Solid Earth, 129, e2023JB028084. https://doi.org/10.1029/2023JB028084 

In [ ]:
# b: healing
def func_healing(independent_vars, params, time_quake="2017,09,19,18,14,00"):
    # full implementation of Snieder's healing model from Snieder et al. 2017
    # but faster
    # (c) by Kurama Okubo
    t = independent_vars[0]
    if len(independent_vars) == 2:
        time_quake = independent_vars[1]

    t_low = t.copy()
    tau_min = 0.1
    tau_max = params[0]
    drop_eq = params[1]
    tquake = UTCDateTime(time_quake).timestamp
    

    tax = t_low - tquake
    ixt = tax > 0
    tax[~ixt] = 0.0

    dv_quake_low = np.zeros(len(tax))

    # separate function accelerated by c and low level callback for scipy quad
    dv_quake_low[ixt] = [logheal_llc(tt, tau_min, tau_max, drop_eq) for tt in tax[ixt]]
    dv_quake_low /= np.log(tau_max/tau_min)
    # reinterpolate
    #f = interp1d(t_low, dv_quake_low, bounds_error=False)
    dv_quake = dv_quake_low  #f(t)
    return(dv_quake)

# for more than 1 quake in the timeseries
def func_healing_list(independent_vars, params):
    t = independent_vars[0]
    quakes = independent_vars[1]

    dv_quakes = np.zeros(len(t))
    tau_max_list = params[0]
    drop_eq_list = params[1]

    for i in range(len(quakes)):
        dv_quakes += func_healing([t], [tau_max_list[i], drop_eq_list[i]], time_quake=quakes[i])
    return(dv_quakes)

### Model for pore pressure effect on dvv

In [ ]:
#################################################
# Hydrology: 1-D poroelastic response to rainfall
#################################################
def get_effective_pressure(rhophi, z, rhos):
    p = np.zeros(len(z))
    dz = np.zeros(len(z))
    dz[:-1] = z[1:] - z[:-1]
    dz[-1] = dz[-2]

    p[1: ] += np.cumsum(rhos * 9.81 * dz)[:-1]  # overburden

    # parameter rhophi: rho water * porosity
    p[1: ] -= z[1:] * rhophi[1:] * 9.81 # roughly estimated pore pressure -- just set to hydrostatic pressure here
    return(p)

def model_SW_dsdp(p_in, waterlevel=100.):
    # 1 / vs  del v_s / del p: Derivative of shear wave velocity to effective pressure
    # identical for Walton smooth model and Hertz-Mindlin model
    # input: 
    # p_in (array of int or float): effective pressure (hydrostatic - pore)
    # waterlevel: to avoid 0 division at the free surface. Note that results are sensitive to this parameter.
    # output: 1/vs del vs / del p

    p_in[p_in < waterlevel] = waterlevel
    sens = 1. / (6. * p_in)
    return(sens)

def roeloffs_1depth(t, rain, r, B_skemp, nu, diff,
                    rho, g, waterlevel, model, nfft=None):
    # evaluate Roeloff's response function for a specific depth r
    # input:
    # t: time vector in seconds
    # rain: precipitation time series in m
    # r: depth in m
    # B_skemp: Skempton's coefficient (no unit)
    # nu: Poisson ratio (no unit)
    # diff: Hydraulic diffusivity, m^2/s
    # rho: Density of water (kg / m^3)
    # g: gravitational acceleration (N / kg)
    # waterlevel: to avoid zero division at the surface. Results are not sensitive to the choice of waterlevel
    # model: drained, undrained or both (see Roeloffs, 1988 paper)
    # output: Pore pressure time series at depth r

    # use nfft to try an increase convolution speed
    if nfft is None:
        nfft = len(t)

    dp = rho * g * rain
    dt = t[1] - t[0]  # delta t, sampling (e.g. 1 day)
    diffterm = 4. * diff * np.arange(len(t)) * dt
    diffterm[0] = waterlevel
    diffterm = r / np.sqrt(diffterm)
    
    resp = erf(diffterm)
    rp = np.zeros(nfft)
    rp[0: len(resp)] = resp
    P_ud = np.convolve(rp, dp, "full")[0: len(dp)]
    
    resp = erfc(diffterm)
    rp = np.zeros(nfft)
    rp[0: len(resp)] = resp
    P_d = np.convolve(rp, dp, "full")[0: len(dp)]
    if model == "both":
        P = P_d + B_skemp * (1 + nu) / (3. - 3. * nu) * P_ud
    elif model == "drained":
        P = P_d
    elif model == "undrained":
        P = B_skemp * (1 + nu) / (3. - 3. * nu) * P_ud
    else:
        raise ValueError("Unknown model for Roeloff's poroelastic response. Model must be \"drained\" or \"undrained\" or \"both\".")
    return P

def roeloffs(t, rain, r, B_skemp, nu, diff, rho=1000.0, g=9.81, waterlevel=1.e-12, model="both"):
    s_rain = np.zeros((len(t), len(r)))
    fftN = next_fast_len(len(t))
    for i, depth in enumerate(r):
        p = roeloffs_1depth(t, rain, depth, B_skemp, nu, diff,
                            rho, g, waterlevel, model, nfft=fftN)
        s_rain[:, i] = p
    return(s_rain)


def func_rain(independent_vars, params):
    # This function does the bookkeeping for predicting dv/v from pore pressure change.
    z = independent_vars[0]
    dp_rain = independent_vars[1]
    rhos = independent_vars[2]
    phis = independent_vars[3]
    kernel = independent_vars[4]

    dz = np.zeros(len(z))
    dz[:-1] = z[1:] - z[:-1]
    dz[-1] = dz[-2]

    waterlevel = params[0]

    rhophi = 1000.0 * phis
    p = get_effective_pressure(rhophi, z, rhos)
    stress_sensitivity = model_SW_dsdp(p, waterlevel)
    dv_rain = np.dot(-dp_rain, stress_sensitivity * kernel * dz)

    return(dv_rain)

### Thermoelastic effect

In [ ]:
#################################################
# Thermoelastic effect following Richter et al., 2015
#################################################

def diff_temp_term(t0_surface, t, z, n, diff, w0=2.*np.pi/(365.25*86400.0)):
    gamma = np.sqrt(n * w0 / (2. * diff))
    ts = t0_surface * np.exp(1.j * (n * t * w0 - gamma * z) - gamma * z)
    return(np.real(ts))

def cn(n, t, y, tau=86400.0 * 365.25):
    c = y * np.exp(-1.j * 2 * n * np.pi * t / tau)
    return c.sum()/c.size


def get_temperature_z(t, T_surface, z, thermal_diffusivity,
                      n_fourier_components=6):
    
    T_surface -= T_surface.mean()

    # get Fourier series representation of temperature
    fcoeffs = np.array([cn(n, t - t.min(), T_surface, tau=86400.0 * 365.25) \
        for n in range(n_fourier_components)])

    # get diffusion result
    difftemp = np.zeros((len(t), len(z)))
    for ix, zz in enumerate(z):
        for n, fc in enumerate(fcoeffs):
            difftemp[:, ix] += np.array([diff_temp_term(fc, tt, zz, n, thermal_diffusivity) \
            for tt in t - t.min()])

    # return diffusion result
    return(difftemp)

def func_temp(independent_vars, params):

    t = independent_vars[0]
    z = independent_vars[1]
    kernel = independent_vars[2]
    dp_temp = independent_vars[3]

    dz = np.zeros(len(z))
    dz[:-1] = z[1:] - z[:-1]
    dz[-1] = dz[-1]

    assert dz[0] > 0.0
    sensitivity_factor = params[0]
    dv_temp = sensitivity_factor * np.dot(dp_temp, kernel * dz)
    return(dv_temp)

### Superposition of several model terms for the final dvv timeseries

In [ ]:
def evaluate_model_quake_rain_temp(ind_vars, params, return_all=False):

    # independent variables in this model: time, depth, surface wave sensitivity kernel (depths equal to depth array)
    # density, porosity, rain in m, temperature in degrees and earthquake integer timestamps of origin time
    t = ind_vars[0]
    z = ind_vars[1]
    z_T = ind_vars[2]
    kernel_vs = ind_vars[3]
    kernel_vs_T = ind_vars[4]
    rho = ind_vars[5]
    phi = ind_vars[6]
    dp_rain = ind_vars[7]
    dp_temp = ind_vars[8]
    quakes_timestamps = ind_vars[9]
    print(quakes_timestamps, len(quakes_timestamps))

    # Parameters: earthquake maximum relaxation times (as many as earthquake timestamps)
    # velocity drops (as many as earthquake timestamps), decadic log of pressure at the surface in Pascal), 
    # decadic log of temperature sensitivity
    
    tau_maxs = [10. ** p for p in params[0: len(quakes_timestamps)]]
    drops = params[len(quakes_timestamps): 2 * len(quakes_timestamps)]
    p0 = 10. ** params[2 * len(quakes_timestamps)]
    tsens = 10. ** params[2 * len(quakes_timestamps) + 1]

    dv_rain = func_rain([z, dp_rain, rho, phi, kernel_vs], [p0])
    dv_temp = func_temp([t, z_T, kernel_vs_T, dp_temp], [tsens])

    dv_quake = np.zeros(len(t))
    for ixq, q in enumerate(quakes_timestamps):
        dv_quake += func_healing([t], [tau_maxs[ixq], drops[ixq]], time_quake=q)
    
    
    return(dv_rain + dv_temp + dv_quake, [dv_rain, dv_temp, dv_quake])

### additional convenience functions

In [ ]:
# Function to replace NaN values with nearest non-NaN values
# by ChatGPT
# prompt: I have a numpy array in Python that contains several NaN-values, and I would like to replace them by the nearest finite value.
def replace_nan_with_nearest(arr, interpolation_method="nearest"):
    indices = np.arange(len(arr))
    valid = ~np.isnan(arr)
    
    # Nearest-neighbor interpolation
    f = interp1d(indices[valid], arr[valid], bounds_error=False,
                 fill_value="extrapolate", kind=interpolation_method)
    
    arr[~valid] = f(indices[~valid])
    return arr


# Start here by loading all the data


#### Load vs sensitivity kernel for Rayleigh waves

In [ ]:
# normalization thickness
normH = 0.015

# Load data
data_dlnC_dlnVs = np.loadtxt('data/1Dsph_mc_Roma_intp_dlnC_dlnVs_RayFund.txt')

m, n = data_dlnC_dlnVs.shape
mT = m - 3
nH = n - 1
period = data_dlnC_dlnVs[2:(mT+1), 0] # period for sk
depth = data_dlnC_dlnVs[0, 1:n]  
middep = (depth[0:(nH-1)] + depth[1:nH]) / 2 # middle depth to plot
thick = depth[1:nH] - depth[0:(nH-1)]
normHratio = normH / thick
dlnC_dlnVs = data_dlnC_dlnVs[2:(mT+1), 1:n-1]

for i in range(nH-1):
    dlnC_dlnVs[:, i] *= normHratio[i]

linestyle_strs = ['solid', 'dashed', 'dotted', 'dashdot', 'solid']
for i_line, i_pd in enumerate([2,3,4]):
    plt.plot(dlnC_dlnVs[i_pd, :], middep, linewidth=2, linestyle=linestyle_strs[i_line], label="Freq = {:.1f} Hz".format(1/period[i_pd])) #

plt.legend()
plt.xlabel('Sensitivity (dlnC/dlnVs)', fontsize=12)
plt.ylabel('Depth')
plt.xlim([-0.005, 0.1])
plt.ylim([1.2, 0])

plt.grid()
plt.show()


# is depth in kilometers?
depth_in_meters = middep * 1000.
K_vs = dlnC_dlnVs[3]  # 0.9 Hz

#### Load density

In [ ]:
vel_df = pd.read_csv('data/1DSph_mc_Roma_intp.txt', delim_whitespace=True)
rho = vel_df["RHO(GM/CC)"].values * 1000.


#### Define the meteoreological data: timestamp, temperature, rain

In [ ]:
# define a function to get meteo data
# needed: Timestamp,  temperature, rain
def get_met_data_yang(plot):
    start_row = 7
    start_date = pd.to_datetime('2022/05/12', format='%Y/%m/%d')
    end_date = pd.to_datetime('2023/02/17', format='%Y/%m/%d')
    
    climate_df = pd.read_csv('data/climatologicas_CEADF.csv', skiprows=range(0, start_row))
    climate_df['Date'] = pd.to_datetime(climate_df['Fecha'], format='%Y/%m/%d')
    climate_df.sort_values(by='Date', inplace=True)
    climate_filtered_df = climate_df[(climate_df['Date'] >= start_date) & (climate_df['Date'] <= end_date)]
    filtered_dates = climate_filtered_df['Date'].values
    climate_keys_list = ['Fecha', ' Precipitación(mm)', ' Temperatura Media(ºC)', ' Temperatura Máxima(ºC)', 
                  ' Temperatura Mínima(ºC)', ' Evaporación(mm)']

    for key in [' Precipitación(mm)', ' Temperatura Media(ºC)',]:
        filtered_para = pd.to_numeric(climate_filtered_df[key], errors='coerce').values.astype('float64') #df[key2plot].values 
        
        if key == " Precipitación(mm)":
            rain = filtered_para
        if key == " Temperatura Media(ºC)":
            Temp_C = filtered_para
            # get rid of nan
            Temp_C = replace_nan_with_nearest(Temp_C, interpolation_method="linear")
        if plot:
            plt.plot(filtered_dates, filtered_para)
            plt.plot(filtered_dates, filtered_para, "--")
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
            plt.ylabel(key)
            plt.xticks(rotation=45)
            plt.show()
    timestamps = [(fd - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') for fd in filtered_dates]
    
    df_out = pd.DataFrame(columns=["dates", "timestamps", "rain", "Temp_C"])
    df_out["dates"] = filtered_dates
    df_out["timestamps"] = timestamps
    
    df_out["rain"] = rain
    df_out["Temp_C"] = Temp_C
    
    #plt.plot(df_interp.timestamps.values[1:], np.diff(df_interp.timestamps.values))
    return(df_out)

df_meteo = get_met_data_yang(plot=True)

#### Load dvv data
needed: timestamps, dvv, correlation coefficient (error is actually only needed for inversion)

In [ ]:
# inputs
ccfreq_ranges = [[0.4, 1.2], [1.2, 3.6]]
freq = ccfreq_ranges[0]
cc_min = 0.6
# load dvv data and define an array of integer timestamps
f_min = freq[0]
if f_min == 0.4: 
    vm_dvv = 1.2
    vm_err = 1
    t_len = 5
elif f_min == 1.2:
    vm_dvv = 0.6
    vm_err = 1
    t_len = 2

file_title = f'data/Source-Receiver-10_Stack-5chans-1days_Tlen-{t_len}s_Freq-{freq[0]:.2f}-{freq[1]:.2f}Hz.npz'
dvv_data_all = np.load(file_title, allow_pickle=True)
# print(list(dvv_data.keys()))

tstamps = np.array([(np.datetime64(ts) - \
            np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's') for ts in dvv_data_all["dates"]])

# replace data below quality criterion
dvv_data_all["dvv"][dvv_data_all["ccs"] < cc_min] = np.nan
dvv_qc = np.zeros(dvv_data_all["dvv"].shape)
for i in range(dvv_data_all["dvv"].shape[0]):
    dvv_qc[i, :] = replace_nan_with_nearest(dvv_data_all["dvv"][i, :])


# All the model parameters go here

In [ ]:
# which effects to consider in the model?

# general inputs
# ==============
consider_effects = ["quake", "temperature"]  # "rain", "temperature", "quake"
ixs = 1050  # source index


# earthquake-related parameters
# =============================
# origin time of the earthquake
qtimes = [UTCDateTime("2022-09-19T18:05:06")]
# list of the log10 of maximum relaxation times (one per earthquake)
log10_tau_maxs = [np.log10(86400. * 365)]
# list of the velocity drops (one per earthquake). Here, 0.1 is 10% velocity drop
drops = [0.07]


# rain-related parameters
# =======================
# undrained Poisson ratio of the material
nu_undrained = 0.45
# hydraulic diffusivity
diff_in = 1.e-4
# parts of Roeloff's model to consider: undrained term, drained term or both
# the undrained term is the pore pressure change due to diffusion, the drained term the elastic load of the rain
# if you choose "both", they are added
roeloffs_terms = "undrained"  # "drained", "undrained" or "both"
B = 1  # Skempton's B
# log10 of the minimum effective pressure. This parameter is the effective pressure at the surface / at the level of the observation.
# hydrostatic pressure - pore pressure
# in N
log10_p0 = np.log10(101000)

# temperature-related parameters
# ===============================
# thermal diffusivity of the soil
diff_in_temp = 1.e-6
# number of Fourier terms to describe the seasonal temperature variation
n_fourier_terms_temperature = 48
# log10 of the sensitivity to thermoelastic changes
log10_tsens = -1.9

### The following cell runs the model for 1 source

In [ ]:
dvv = dvv_qc[ixs, :]
f = interp1d(df_meteo.timestamps.values, df_meteo.rain.values, bounds_error=False, fill_value="extrapolate")
rain_m = f(tstamps) / 1000.
rain_m -= np.mean(rain_m)
f = interp1d(df_meteo.timestamps.values, df_meteo.Temp_C.values, bounds_error=False, fill_value="extrapolate")
temperature_C = f(tstamps)

dp_rain = roeloffs(tstamps, rain_m, depth_in_meters, B, nu_undrained, diff_in, model=roeloffs_terms)

# for temperature we need a finer depth sampling, as it affects mostly the upper meters
# for the chosen model
depth_in_meters_T = np.concatenate([np.arange(0, depth_in_meters[2] + 0.5, 0.5), depth_in_meters[3:]])
dp_temp = get_temperature_z(tstamps, 0.2 * temperature_C, depth_in_meters_T,
diff_in_temp, n_fourier_components=n_fourier_terms_temperature)

# uncomment below to plot some rough sketches of the pore pressure effect

# xv, yv = np.meshgrid(tstamps, -depth_in_meters, indexing="ij")
# plt.pcolormesh(xv, yv, dp_rain)
# plt.xticks(rotation=45)
# xt = plt.xticks()
# plt.xticks(xt[0][1:-1], [UTCDateTime(xtt).strftime('%Y-%m') for xtt in xt[0]][1:-1])
# plt.ylim(-500, 0)
# plt.show()

# plt.plot(tstamps, rain_m * 1000. / 60.)
# plt.plot(tstamps, dp_rain[:, 0] / dp_rain.max())
# plt.legend(["rain normalized", "pore pressure"])

#uncomment below to plot some rough sketches of the thermoelastic effect
# xv, yv = np.meshgrid(tstamps, -depth_in_meters_T, indexing="ij")
# print(dp_temp.max(), dp_temp.min())
# plt.pcolormesh(xv, yv, dp_temp)
# plt.colorbar()
# plt.ylim(-30, 0)
# print(np.abs(dp_temp).min())


# independent variables in this model: time, depth, surface wave sensitivity kernel (depths equal to depth array)
# surface wave sensitivity kernel for temperature effect (depths equal to temperature depth array refined at top)
# density, porosity, rain in m, temperature in degrees and earthquake integer timestamps of origin time
f = interp1d(depth_in_meters, K_vs, bounds_error=False, fill_value="extrapolate", kind="nearest")
K_vs_T = f(depth_in_meters_T)
phi = np.ones(len(rho)) * 0.15
independent_variables = [tstamps, depth_in_meters, depth_in_meters_T, K_vs, K_vs_T, rho, phi, dp_rain, dp_temp, qtimes]

parameters = [*log10_tau_maxs, *drops, log10_p0, log10_tsens]
print(parameters)

results = evaluate_model_quake_rain_temp(independent_variables, params=parameters)
effect_rain, effect_temp, effect_quake = results[1]

### The following cell plots the data and model results for 1 source and saves results to a csv

In [ ]:
model = np.zeros(len(tstamps))
if "rain" in consider_effects:
    model += effect_rain * 100.  # in % 
if "temperature" in consider_effects:
    model += effect_temp * 100.  # in %
if "quake" in consider_effects:
    model+= effect_quake * 100.  # in %


plt.figure(figsize=(12, 4))
plt.plot(tstamps, model , linewidth=1.5, label=f"model {[p for p in consider_effects]}")
plt.plot(tstamps, dvv_qc[ixs, :], "k", linewidth=1.5, label="observed dv/v")

# you can also plot the single terms
#plt.plot(tstamps, effect_rain * 100,  alpha=0.5, label="precipitation effect")
#plt.plot(tstamps, effect_temp * 100, alpha=0.5, label="thermoelastic effect")
#plt.plot(tstamps, effect_quake * 100, alpha=0.5, label="earthquake drop & healing")

plt.xticks(rotation=45)
xt = plt.xticks()
plt.xticks(xt[0][1:-1], [UTCDateTime(xtt).strftime('%Y-%m') for xtt in xt[0]][1:-1])
plt.ylabel("dv/v / %")
plt.legend()

df_out = pd.DataFrame(columns=["timestamps", "dvv_observed", "dvv_precipitation", "dvv_temperature", "dvv_earthquakes"])
df_out["timestamps"] = tstamps
df_out["dvv_observed"] = dvv_qc[ixs, :]
df_out["dvv_precipitation"] = effect_rain
df_out["dvv_temperature"] = effect_temp
df_out["dvv_earthquakes"] = effect_quake

df_out.to_csv(f"result_source{ixs}.csv")